In [102]:
import PreProcessing
from Prediction import AIPrediction
import pandas

In [103]:
#Define Prediction name
Pname='LinearRegression'

#Select starting variables. If you want to start again, make sure to erase 'Good_variables_%s.json'%Pname 
Good_variables={'pCR': [u'KDR_Squared', u'ATF3', u'RPS6_Squared', u'cyto.cat=Misc', u'GATA3', u'CDKN2A_Squared', u'NF2.pS518', u'CASP9.cl315_Squared', u'IGFBP2', u'SMAD3_Squared', u'PRKAA1_2.pT172_Squared', u'HDAC3_Squared', u'CLPP', u'PRIOR.MAL', u'ATG7', u'cyto.cat=diploid', u'DLX1_Squared', u'MSI2', u'CCNE2', u'NPM1.3542', u'ARC', u'cyto.cat=21', u'ITGAL', u'SMAD2_Squared', u'RPS6.pS240_244', u'MYC', u'LCK_Squared', u'ITGA2', u'GAPDH', u'CCNE1', u'PA2G4.pT70_Squared', u'cyto.cat=-7', u'MTOR.pS2448_Squared', u'CD44', u'PRKCB.II_Squared', u'MAP2K1_2.pS217_221_Squared', u'BAD.pS136_Squared', u'CASP9.cl330', u'GSKA_B.pS21_9', u'CTSG', u'FOXO3_Squared', u'TGM2', u'STAT3.pS727', u'CASP8_Squared', u'PIK3CA', u'RPS6', u'SFN', u'PTK2_Squared', u'ZNF296_Squared', u'PRKCD.pT507', u'Age.at.Dx', u'STMN1_Squared', u'YWHAZ_Squared', u'HSPB1', u'STMN1', u'PDK1.pS241_Squared', u'CDK1', u'MAPK9'],
                'pRelapse': [u'cyto.cat=t9;22', 'IGFBP2_Squared', u'CCND3', u'KIT_Squared', u'PTEN.pS380T382T383', u'BCL2_Squared', u'BAK1_Squared', u'SMAD5.pS463_Squared', 'MDM2', 'ARC', u'PTPN11_Squared', u'H3histon_Squared', u'PA2G4.pS65_Squared', 'HDAC1_Squared', u'EIF2S1.pS51._Squared'], 
                'OS': [u'PRIOR.MAL', u'ARC', u'cyto.cat=diploid', u'H3histon', u'Age.at.Dx', u'PTGS2_Squared', u'SMAD4', u'PA2G4.pS65', u'STMN1', u'EIF2AK2', u'H3K27Me3', u'HSP90AA1_B1'], 
                'Remission': [u'CASP9.cl330', u'ERG', u'ALBUMIN', u'CASP3.cl175', u'TP53', u'RPS6KB1.pT389', u'PLAC1', u'JMJD6', u'SMAD3_Squared', u'ERG_Squared', u'TRIM24', u'Age.at.Dx', u'HSPA1A_L', u'ATG7_Squared', u'ARC_Squared', u'STAT3.pS727', u'CBL_Squared', u'BIRC5_Squared', u'ARC', u'YWHAE', u'SMAD5.pS463', u'BRAF_Squared', u'MTOR.pS2448_Squared']}

#Select Dependent Variables
Variables_for_prediction={'pCR':'resp.simple',
                          'pRelapse':'Relapse',
                          'OS':'Overall_Survival_cut',
                          'Remission':'Remission_Duration_cut'}

#Select scoring measure to use for minimization
Fast_measure='BAC' #Select between BAC PCC or Scr. You can select two Measures, 
                       #the second measure will overwrite the first one. ('BAC','Auroc')
Slow_measure={'pCR':'BAC',
              'pRelapse':'BAC',
              'OS':'BAC',
              'Remission':'BAC'}

#Select max time in hours to run the prediction
max_time_h= 8#Max number of hours that you can run the simulation

In [104]:
#Read prediction bias
Var_value=pandas.DataFrame.from_csv('Variable_value.csv')

In [ ]:
#Define prediction class
from sklearn import linear_model
Pred=AIPrediction(pivot=Good_variables,method=linear_model.LinearRegression())
print Pred.result(rep=10)

              AUROC       BAC        CI       PCC       Scr
pCR        0.949485  0.852799       NaN  0.884548 -2.454186
pRelapse   0.766794  0.664629       NaN  0.717755 -2.941280
OS              NaN  0.441962  0.685279  0.713083 -3.709842
Remission       NaN  0.460416  0.686021  0.740003 -5.313586

[4 rows x 5 columns]


In [ ]:
#Define Best Prediction
Pred.pivot=Good_variables
groups=Pred.create_groups(50)
Best_scores=Pred.score(groups=groups,Measure=tuple(set(Slow_measure.values())))
Best_result=Pred.accuracy(groups=groups,Measure=Fast_measure)

In [ ]:
if True:
    #Open Good variables if it exists
    try:
        with open('Good_variables_%s.json'%Pname) as handle:
            Good_variables=json.loads(handle.read())
    except IOError:
        print 'Did not found previous good variables'
        pass
    #Open Variable values if it exists
    try:
        pandas.DataFrame.from_csv('Var_value_%s.csv'%Pname)
    except IOError:
        print 'Did not found previous variable values'
        pass

In [ ]:
#Define Biased Sample
import random,numpy
def biased_sample(variables,k,bias):
    var=list(variables)
    Bias=bias[variables].copy()
    n = len(variables)
    bias=numpy.cumsum(Bias[variables])
    if len(bias)==0:
        bias=[1 for i in range(n)]
    if not 0 <= k <= n:
        #print population
        k=n
        #raise ValueError("sample larger than population")
    r = random.random()
    result = [None] * k # Initialize result
    selected = set()
    max_freq=bias[-1]
    for i in xrange(k):
        #print i
        r = random.random() * max_freq
        #r=max_freq
        for j,b in enumerate(bias):
            #print r,j,b
            if r<b:
                break
        selected.add(var[j])
        result[i] = var[j]
        #print r,b,j,var[j],'\n',bias
        var.pop(j)
        bias=numpy.cumsum(Bias[var])
        if len(bias)>0:
            max_freq=bias[-1]
        else:
            break
    return result

In [ ]:
#Evolution algorithm
import time,json

Stop=time.time()+60*60*max_time_h #Stop time in seconds
i=0 #Iteration number counter
while time.time()<Stop:  
    Update_Best=False #Only update best if conditions apply
    i=i+1
    #print i
    #Select Test variables
    Test_variables={}
    change_variables={}
    change={}
    for key in Good_variables.keys():
        Rnd=random.random()
        if Rnd>0.65: #Sometimes drop a variable
            l=len(Good_variables[key])
            n=min(l-random.randint(1,5),1)
            change.update({key:'remove'})
            new_variables=biased_sample(Good_variables[key],n,1/(Var_value[key]+1))
            change_variables.update({key:[v for v in Good_variables[key] if v not in new_variables]})
            Test_variables.update({key:new_variables})
        elif Rnd>0.1: #Sometimes add a variable
            varis=[v for v in Var_value[key].index if v not in Good_variables[key]]
            l=len(varis)
            n=min(random.randint(1,5),l)
            change.update({key:'add'})
            change_variables.update({key:biased_sample(varis,n,Var_value[key])})
            new_variables=Good_variables[key]+change_variables[key]
            Test_variables.update({key:new_variables})
        else: #Sometimes choose something completely random
            l=min(len(Var_value[key].index),len(Good_variables[key])*2)
            change.update({key:'try'})
            new_variables=biased_sample(varis,random.randint(1,l),Var_value[key])
            change_variables.update({key:new_variables})
            Test_variables.update({key:new_variables})
        #print key,change, change_variables[key]

    #Update Test_variables
    Pred.pivot=Test_variables
    Test_result=Pred.accuracy(groups=groups[:10],Measure=Fast_measure) #Fast measure
    
    #Test against best prediction
    Next=True
    rew={}
    for key in Good_variables.keys():
        rew.update({key:1.0})
        if Test_result[key]>Best_result[key]:
            Next=False
            if change[key]=='add':
                rew.update({key:1.05})
            elif change[key]=='remove':
                rew.update({key:0.95})
            elif change[key]=='try':
                rew.update({key:1.02})
            else:
                raise 'Undefined change'       
        else:
            if change[key]=='add':
                rew.update({key:0.9})
            elif change[key]=='remove':
                rew.update({key:1.1})
            elif change[key]=='try':
                rew.update({key:0.98})
            else:
                raise 'Undefined change'
    
    #Update bias weights
    for key in Good_variables.keys():
        for var in change_variables[key]:
            if Var_value[key][var]>0.02 and Var_value[key][var]<2.0:
                Var_value[key][var]*=rew[key]
            if Var_value[key][var]<0.02:
                Var_value[key][var]=0.02
            if Var_value[key][var]>2.0:
                Var_value[key][var]=2.0
    #Write to csv
    Var_value.to_csv('Var_value_%s.csv'%Pname)
    
    #Continue with next if it did not improve
    if Next:
        continue
    
    #Test how many times it has a better result
    Test_scores=Pred.score(groups=groups,Measure=tuple(set(Slow_measure.values())))
    for key in Test_scores.keys():
        conf=0.0
        new_BACs=Test_scores[key][Slow_measure[key]]
        old_BACs=Best_scores[key][Slow_measure[key]]
        new_BACs.sort()
        old_BACs.sort()
        pos=0.0
        tot=float(len(new_BACs)**2)
        for new in new_BACs:
            for k,old in enumerate(old_BACs):
                if old>new:
                    pos+=k
                    break
        conf=pos/tot
        #If more than 60% are better, update
        if conf>0.6:
            with open('Log_%s.txt'%Pname,'a+') as handle:
                handle.write('Updated: '+key+'\n')
            Good_variables.update({key:Test_variables[key]})
            Update_Best=True
    
    #Update variables
    if Update_Best:
        Pred.pivot=Good_variables
        groups=Pred.create_groups(50)
        Best_result=Pred.accuracy(groups=groups)
        Best_scores=Pred.score(groups=groups,Measure=tuple(set(Slow_measure.values())))
        with open('Log_%s.txt'%Pname,'a+') as handle:
            handle.write(str(Good_variables)+'\n')
            handle.write(str(Best_result)+'\n')
        with open('Good_variables_%s.json'%Pname,'w+') as handle:
            handle.write(json.dumps(Good_variables))

In [ ]:
print 'iter',i
Pred.pivot=Good_variables
print Pred.result(rep=10)

In [99]:
for key in Test_result.keys():
    conf=0.0
    new_BACs=Test_scores[key][Slow_measure[key]]
    old_BACs=Best_scores[key][Slow_measure[key]]
    new_BACs.sort()
    old_BACs.sort()
    pos=0.0
    tot=float(len(new_BACs)**2)
    for new in new_BACs:
        for k,old in enumerate(old_BACs):
            if old>new:
                pos+=k
                break
    conf=pos/tot
    print key,conf

Remission 0.480096
pCR 0.501424
pRelapse 0.452704
OS 0.248656


In [33]:
import pandas,numpy
Independent=[v for v in PreProcessing.Q_training if v not in PreProcessing.Q_Dependent]
Pool          =pandas.DataFrame({'pCR': [1.0 for v in Independent],
                'pRelapse': [1.0 for v in Independent], 
                'OS': [1.0 for v in Independent], 
                'Remission': [1.0 for v in Independent]}, index=Independent)
Pool.to_csv('Variable_value.csv')
numpy.cumsum(Pool['OS'].values)


909

In [45]:
Test_result,Best_result

({'OS': 0.34173106377053747,
  'Remission': 0.33480392156862748,
  'pCR': 0.45628547250961055,
  'pRelapse': 0.63712699473728895},
 {'OS': {'BAC': [0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.35416666666666669,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.31481481481481483,
    0.33333333333333331,
    0.33333333333333331,
    0.2857142857142857,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.35416666666666669,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.35294117647058826,
    0.40211640211640209,
    0.33333333333333331,
    0.33333333333333331,
    0.33333333333333331,
    0.3

In [129]:
Best_scores['OS'].keys()

['PCC', 'CI', 'BAC', 'Scr']

In [99]:
import random

population=[u'cyto.cat=t9;22', 'IGFBP2_Squared', u'CCND3', u'KIT_Squared', u'PTEN.pS380T382T383', u'BCL2_Squared', u'BAK1_Squared', u'SMAD5.pS463_Squared', 'MDM2', 'ARC', u'PTPN11_Squared', u'H3histon_Squared', u'PA2G4.pS65_Squared', 'HDAC1_Squared', u'EIF2S1.pS51._Squared']
bias=numpy.cumsum(Pool['pCR'][population])
k=5
def biased_sample(variables,k,bias):
    n = len(population)
    bias=numpy.cumsum(bias[population])
    if len(bias)==0:
        bias=[1 for i in range(n)]
    if not 0 <= k <= n:
        #print population
        k=n
        #raise ValueError("sample larger than population")
    r = random.random()
    result = [None] * k # Initialize result
    selected = set()
    max_freq=bias[-1]
    for i in xrange(k):
        #print i
        r = random.random() * max_freq
        for j,b in enumerate(bias):
            #print r,j,b
            if r<b:
                break
        while j in selected:
            r = random.random() * max_freq
            for j,b in enumerate(bias):
                #print r,j,b
                if r<b:
                    break
        selected.add(j)
        result[i] = population[j]
    return result
biased_sample(population,1,Pool['pCR'])

[u'PTPN11_Squared']

In [44]:
population=[u'cyto.cat=t9;22', 'IGFBP2_Squared', u'CCND3', u'KIT_Squared', u'PTEN.pS380T382T383', u'BCL2_Squared', u'BAK1_Squared', u'SMAD5.pS463_Squared', 'MDM2', 'ARC', u'PTPN11_Squared', u'H3histon_Squared', u'PA2G4.pS65_Squared', 'HDAC1_Squared', u'EIF2S1.pS51._Squared']
bias=Pool['pCR']

In [75]:
bias=numpy.cumsum(Pool['pCR'][population])

<bound method Series.pop of cyto.cat=t9;22           1
IGFBP2_Squared           2
CCND3                    3
KIT_Squared              4
PTEN.pS380T382T383       5
BCL2_Squared             6
BAK1_Squared             7
SMAD5.pS463_Squared      8
MDM2                     9
ARC                     10
PTPN11_Squared          11
H3histon_Squared        12
PA2G4.pS65_Squared      13
HDAC1_Squared           14
EIF2S1.pS51._Squared    15
Name: pCR, dtype: int64>

In [54]:
import random
random

<module 'random' from '/usr/lib/python2.7/random.pyc'>

In [101]:
Pool.to_csv('Variable_value.csv')

In [65]:
r

8.583354201546422

In [28]:
import pandas,numpy
Independent=[v for v in PreProcessing.Q_training if v not in PreProcessing.Q_Dependent]
Pool          =pandas.DataFrame({'pCR': [1.0 for v in Independent],
                'pRelapse': [1.0 for v in Independent], 
                'OS': [1.0 for v in Independent], 
                'Remission': [1.0 for v in Independent]}, index=Independent)
Pool.to_csv('Variable_value.csv')